# Определение тональности текста

Используется выборка с отзывами на фильмы [sentence polarity dataset](http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz)
В ней содержатся предложения с отрицательными и положительными отзывами (по 5331).

In [1]:
import os
import re
import collections, itertools, functools
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews, stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

POLARITY_DATA_DIR = os.path.join('polarityData', 'rt-polaritydata')
RT_POLARITY_POS_FILE = os.path.join(POLARITY_DATA_DIR, 'rt-polarity-pos.txt')
RT_POLARITY_NEG_FILE = os.path.join(POLARITY_DATA_DIR, 'rt-polarity-neg.txt')

from __future__ import division

Изначальная модель: наивный Байес. Каждому предложению сопоставляется бинарный bag-of-words вектор.

In [2]:
def use_all(words):
    return dict([(word, True) for word in words])
 
def getWords(file_path, type_of_feature=None, feature_select=None, apply_features=True):
    features = []
    with open(file_path, 'r') as sentences:
        for sentence in sentences:
            words = re.findall(r"[\w']+|[.,!?;]", sentence.rstrip())
            if apply_features:
                words = [feature_select(words), type_of_feature]
            features.append(words)
    return features

def evaluate(feature_selector, show_features=False):
    pos_features = getWords(RT_POLARITY_POS_FILE, 'pos', feature_select=feature_selector)
    neg_features = getWords(RT_POLARITY_NEG_FILE, 'neg', feature_select=feature_selector)
 
    train_fraction = int(len(pos_features)*3/4)
    
    train_features = neg_features[:train_fraction] + pos_features[:train_fraction]
    test_features = neg_features[train_fraction:] + pos_features[train_fraction:]
    clf = NaiveBayesClassifier.train(train_features)
    actual_labels = collections.defaultdict(set)
    pred_lables = collections.defaultdict(set)
    for num, (features, label) in enumerate(test_features):
            actual_labels[label].add(num)
            observed = clf.classify(features)
            pred_lables[observed].add(num)
 
    print 'accuracy:', nltk.classify.util.accuracy(clf, test_features)
    print 'precision:', nltk.metrics.precision(actual_labels['pos'], pred_lables['pos'])
    print 'recall:', nltk.metrics.recall(actual_labels['pos'], pred_lables['pos'])
    if show_features:
        clf.show_most_informative_features()
 


In [3]:
evaluate(use_all)

accuracy: 0.773068267067
precision: 0.787066246057
recall: 0.748687171793


Попробуем повысить качество модели. Вначале уберем стоп-слова

In [4]:
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
 
def stopword_filtered_word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])
 
evaluate(stopword_filtered_word_feats)

accuracy: 0.764066016504
precision: 0.765060240964
recall: 0.762190547637


Качество только ухудшилось, следовательно стоп-слова помогают в определении тональности. Попробуем добавить в признаки самые значимые биграммы.

In [5]:
def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=50):
    all_bigrams = BigramCollocationFinder.from_words(words)
    bigrams = all_bigrams.nbest(score_fn, n)
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])

evaluate(bigram_word_feats)

accuracy: 0.781695423856
precision: 0.791311093871
recall: 0.765191297824


Качество немного выросло, теперь попробуем понизить размерность выборки, используя хи-квадрат метрику.

In [6]:
def get_best_words(num=10000):
    word_freq = FreqDist()
    label_freq = ConditionalFreqDist()

    for word in itertools.chain(*getWords(RT_POLARITY_POS_FILE, 'pos', apply_features=False)):
        word_freq[word.lower()] += 1
        label_freq['pos'][word.lower()] += 1

    for word in  itertools.chain(*getWords(RT_POLARITY_NEG_FILE, 'neg', apply_features=False)):
        word_freq[word.lower()] += 1
        label_freq['neg'][word.lower()] += 1

    pos_word_count = label_freq['pos'].N()
    neg_word_count = label_freq['neg'].N()
    total_word_count = pos_word_count + neg_word_count

    word_scores = {}

    for word, freq in word_freq.iteritems():
        pos_score = BigramAssocMeasures.chi_sq(label_freq['pos'][word],
            (freq, pos_word_count), total_word_count)
        neg_score = BigramAssocMeasures.chi_sq(label_freq['neg'][word],
            (freq, neg_word_count), total_word_count)
        word_scores[word] = pos_score + neg_score

    best = sorted(word_scores.iteritems(), key=lambda (w,s): s, reverse=True)[:num]
    bestwords = set([w for w, s in best])
    return bestwords
 
def best_word_feats(words, bestwords):
    return dict([(word, True) for word in words if word in bestwords])
 
evaluate(functools.partial(best_word_feats, bestwords=get_best_words()))

accuracy: 0.846586646662
precision: 0.867834394904
recall: 0.817704426107


Теперь качество улучшилось более значительно, посмотрим на ключевые слова, позволяющие определить тональность.

In [7]:
evaluate(functools.partial(best_word_feats, bestwords=get_best_words()), show_features=True)

accuracy: 0.846586646662
precision: 0.867834394904
recall: 0.817704426107
Most Informative Features
              engrossing = True              pos : neg    =     17.0 : 1.0
                   quiet = True              pos : neg    =     15.7 : 1.0
                mediocre = True              neg : pos    =     13.7 : 1.0
               absorbing = True              pos : neg    =     13.0 : 1.0
                portrait = True              pos : neg    =     12.4 : 1.0
                   flaws = True              pos : neg    =     12.3 : 1.0
              refreshing = True              pos : neg    =     12.3 : 1.0
               inventive = True              pos : neg    =     12.3 : 1.0
                 triumph = True              pos : neg    =     11.7 : 1.0
            refreshingly = True              pos : neg    =     11.7 : 1.0
